# Supporting functions

In [123]:
import pandas as pd
import operator
import re
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import json

In [2]:
# function for common string cleanup
def cleanup(x):
    return str(x).replace("\t","").replace("\s","").strip()

# Reading mater drug file and creating dictionary
returning relevent result to the entered one

In [3]:
df = pd.read_csv("data/out_unique_final.csv", sep="ж")
df = df.apply(lambda x: x.astype(str).str.lower())

/home/sunil.patel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df.columns

Index(['S/N', 'Manufacturer', 'Name', 'Type', 'Unit(Dose)',
       'Constituents/Unit', 'Package unit', 'Price(In Rs.)',
       'Price/Unit (In Rs.)'],
      dtype='object')

In [5]:
class master_drug_file_handeller:
    max_replacements = 10
    def __init__(self):
        self.master_drug_list = []
        self.master_drug_store = []
        for i in range(0, len(df)):
            drug_dict = df.iloc[i].to_dict()
            drug_dict['id'] = i
            self.master_drug_list.append(drug_dict['Constituents/Unit'])
            self.master_drug_store.append(drug_dict)
    
    def relevent_searcher(self, text):
        self.relevence_dict = {}
        scores = [fuzz.ratio(i, text) for i in self.master_drug_list]
        scores_ids = [i for i in range(0,len(self.master_drug_list))]
        for score,id in zip(scores, scores_ids):
            if id not in self.relevence_dict.keys():
                self.relevence_dict[id] = score
        #soriting the suggestion    
        sorted_x = sorted(self.relevence_dict.items(), key=operator.itemgetter(1), reverse=True)
        return  sorted_x[:10]

In [6]:
MDFH = master_drug_file_handeller()

In [7]:
relevence_dict_list = MDFH.relevent_searcher("PIPERACILLIN+TAZOBACTUM".lower())

In [8]:
MDFH.master_drug_store[37032]

{'Constituents/Unit': 'piperacillin+tazobactam',
 'Manufacturer': 'profic organic ltd.',
 'Name': 'proact (4000+500)',
 'Package unit': '1 vial',
 'Price(In Rs.)': '189.00',
 'Price/Unit (In Rs.)': '189.00',
 'S/N': '7',
 'Type': 'injection',
 'Unit(Dose)': '1 vial',
 'id': 37032}

# Reading file provided by Janki - Patient Data

---

### Supporting function

In [25]:
import random
import time
import string
from pandas import Series

In [56]:
def generate_random_age(num):
    standard_limit = random.randint(-5,5)
    random_age = num+standard_limit
    return int(random_age)

In [11]:
def generate_random_patient_Name(stringLength=2):
    """
    generate_random_patient_Name
    """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength)).upper()

In [12]:
def generate_sex():
    """
    generate_sex()
    """
    random_prob = random.random()
    if random_prob > 0.66:
        return 2
    else: 
        return 1

In [64]:
def generate_weight(num):
    """
    generate_weight(num)
    """
    random_prob = random.random()
    if random_prob >= 0.1  and  random_prob <= 0.7 :
        # retuen None as many of the values are none
        return None
    else:
        standard_limit = random.randint(-5,5)
        random_age = num+standard_limit
        return int(random_age)


In [67]:
def generate_weight():
    "always not recorded"
    return None
def generate_BMI():
    "always not recorded"
    return None
def generate_antibiotics_in_past():
    """
    Assuming 20% time patient dont know or taking patients taking antibiotics for the first time
    """
    random_prob = random.random()
    if random_prob < 0.4:
        # 0.1 % person knws tahat they have takenn antibiotics in the past
        return "Yes"
    if random_prob >= 0.4  and  random_prob <= 0.9 :
        # not sure patients
        return None
    elif random_prob >= 0.9 :
        # 0.1 % person knws tahat they have  not takenn antibiotics in the past
        return "No"
def generate_details_of_antibiotics_used():
    """
    Always render no as no one rememberes waht was taken last time
    """
    return "Dont Know"

def generate_num_day_hospitalized(min, max):
    """
    randomly generatiing number of days stayed in hospital  
    """
    return random.randint(min, max)
def generate_culture_sensitivity_test():
    random_prob = random.random()
    if random_prob>0.90:
        return "performed"
    else:
        return "Not performed"


### Antibiotic group genration

In [15]:
# getting/antibiotic_class
antibiotic_mapping = pd.read_excel('data/antibiotic_class.xlsx',sheetname='Sheet1')

In [16]:
antibitic_class_mapping = {}
for i in range(0,len(antibiotic_mapping)):
    dict_ = antibiotic_mapping.iloc[i].to_dict()
    antibitic_class_mapping[dict_['Drug Name'].lower().replace("/",'+').strip()] = dict_['Drug class'].lower().strip()

In [128]:
def generate_details_for_antibiotics(antibiotic_generic_name):
    relevence_dict_list = MDFH.relevent_searcher(antibiotic_generic_name.lower())
    print(relevence_dict_list)
    random_chosen  = random.randint(0, len(relevence_dict_list))
    random_antbiotic_id  = relevence_dict_list[random_chosen][0]
    return MDFH.master_drug_store[random_antbiotic_id]

def get_antibiotic_class(antibiotic):
    if '+' in antibiotic:
        return "Fixed dose combiination"
    try:
        class_  = antibitic_class_mapping[str(antibiotic).lower()]
        return class_
    except:
        return "**NA**"
def get_dose_route(form):
    if 'tablet' in form.lower() or 'capsule'  in form.lower():
        return "Oral"
    else:
        return "IV"

def get_dosage(text_with_dosage):
    found =  re.findall(r'(\d+){1,3}', text_with_dosage, re.I)
    return [int(i) for i in found]
    
def generate_antibiotic_group(generic_name,group_no=1, new_dict = {}):
    """
     '(Antibiotic 1) Generic Name ',
     '(Antibiotic 1) Class',
     '(Antibiotic 1) Dose (mg)',
     '(Antibiotic 1) Route',
     '(Antibiotic 1) Units',
     '(Antibiotic 1) Total Dose Administered (mg)',
     '(Antibiotic 1) Cost (Rs)',
     '(Antibiotic 1) Total Cost (Rs)',
     New info 
     {'Constituents/Unit': 'piperacillin+tazobactam',
     'Manufacturer': 'profic organic ltd.',
     'Name': 'proact (4000+500)',
     'Package unit': '1 vial',
     'Price(In Rs.)': '189.00',
     'Price/Unit (In Rs.)': '189.00',
     'S/N': '7',
     'Type': 'injection',
     'Unit(Dose)': '1 vial',
     'id': 37032}
    """
    dict_ = dict(generate_details_for_antibiotics(generic_name))
#     print(dict_)
    new_dict['(Antibiotic '+str(group_no)+') Generic Name'] =  dict_['Constituents/Unit']
    new_dict['(Antibiotic '+str(group_no)+') Class'] =  get_antibiotic_class(generic_name)
    new_dict['(Antibiotic '+str(group_no)+') Dose (mg)'] = get_dosage(dict_['Name'])
    new_dict['(Antibiotic '+str(group_no)+') Route'] = get_dose_route(dict_['Package unit'])
    units_administered = new_dict['(Antibiotic '+str(group_no)+') Units'] =  random.randint(3,12)
    new_dict['(Antibiotic '+str(group_no)+') Total Dose Administered (mg)'] = [i*units_administered for i in get_dosage(dict_['Name'])]
    new_dict['(Antibiotic '+str(group_no)+') Cost (Rs)'] = float(dict_['Price/Unit (In Rs.)'])
    new_dict['(Antibiotic '+str(group_no)+') Total Cost (Rs)'] =  float(dict_['Price/Unit (In Rs.)'])*units_administered

    return new_dict

In [129]:
generate_antibiotic_group("cefixime", 3)

[(13216, 100), (13217, 100), (13218, 100), (13219, 100), (13220, 100), (13221, 100), (13222, 100), (13223, 100), (13225, 100), (13226, 100)]


{'(Antibiotic 3) Class': 'cephalosporin',
 '(Antibiotic 3) Cost (Rs)': 0.9,
 '(Antibiotic 3) Dose (mg)': [100],
 '(Antibiotic 3) Generic Name': 'cefixime',
 '(Antibiotic 3) Route': 'Oral',
 '(Antibiotic 3) Total Cost (Rs)': 4.5,
 '(Antibiotic 3) Total Dose Administered (mg)': [500],
 '(Antibiotic 3) Units': 5}

---

In [130]:
patient_data = pd.read_excel('data/xcel_patient_data.xlsx',sheetname='Sheet1&3')

In [131]:
print(list(patient_data.columns))

['Form No.', 'Hosp. Name', 'Pt. Name/ IP no', 'Sex', 'Age (in yr)', 'Weight (in kg)', 'BMI (kg/m2)', 'Antibiotic in Past', 'If Yes, Details of Antibiotic used', 'No. of Days Hospitalized', 'Diagnosis', 'Culture SensitIVity Test', 'No. of Drugs/prescription', 'No. of Antibiotics', 'No. of Antibiotics by Generic Name', 'No. of Antibiotics by Brand Name', 'No. of FDC', 'No. of Antibiotics prescribed from NLEM 2017', 'Switch Over therapy', 'Parentral', 'Oral', 'Syrup', 'Eye drop', 'Ear drop', 'Others formulations', '(Antibiotic 1) Generic Name ', '(Antibiotic 1) Class', '(Antibiotic 1) Dose (mg)', '(Antibiotic 1) Route', '(Antibiotic 1) Units', '(Antibiotic 1) Total Dose Administered (mg)', '(Antibiotic 1) Cost (Rs)', '(Antibiotic 1) Total Cost (Rs)', '(Antibiotic 2) Generic Name', '(Antibiotic 2) Class', '(Antibiotic 2) Dose (mg)', '(Antibiotic 2) Route', '(Antibiotic 2) Units', '(Antibiotic 2) Total Dose Administered (mg)', '(Antibiotic 2) Cost (Rs)', '(Antibiotic 2) Total Cost (Rs)', '(

---
# Mater iterator

In [132]:
multiplication_factor = 10
random_pic_list = ["amikacin", "amoxicilin+clavulanic acid", "amoxicillin", "amoxicillin + clavulanicacid", "amoxicillin+clavulanic acid", "amoxicilln", "azithromycin", "cefepime", "cefixime", "cefixime + salbutamol", "cefoparazone+salbactum", "cefopaxime", "cefotaxime", "cefotaxime + salbactum", "ceftriaxone", "ceftriaxone + salbactum", "ceftriaxone+salbactum", "cetriaxone", "ciprofloxacil", "ciprofloxacin", "clarithromycin", "clindamycin", "doxicycllin", "doxycyclin", "doxycycline", "levofloxacin", "meropenem", "metronidaxole", "metronidazole", "moxifloxacin", "ofloxacin", "ofloxcin", "piperacillin", "piperacillin+tazobactum"]

In [133]:
def get_new_antibiotics(existing_antibiotic):
    for each_existing_antibiotic in existing_antibiotic:
        if each_existing_antibiotic.lower() not in random_pic_list:
            return each_existing_antibiotic.lower()
    
def generate_antibiotic_groups(modified_dict):
    total_groups = 7 
    num_gropus = random.randint(1,7)
    for group_no in range(1,num_gropus):
        existing_antibiotic = modified_dict['(Antibiotic '+str(group_no)+') Generic Name']
        if existing_antibiotic != 'NaN':
            modified_dict = generate_antibiotic_group(existing_antibiotic,group_no=group_no, new_dict=modified_dict)
        else:
            new_antibiotic = get_new_antibiotics(existing_antibiotic)
            modified_dict = generate_antibiotic_group(new_antibiotic,group_no=group_no, new_dict=modified_dict)
            break
    return modified_dict

In [134]:
for line_no in range(0, len(patient_data)):
    for fake_gen_no in (0, multiplication_factor):
        original_dict = patient_data.iloc[0].to_dict()
        modified_dict = original_dict
        ################################################
        modified_dict['Pt. Name/ IP no'] = generate_random_patient_Name()
        modified_dict['Sex'] = generate_sex()
        modified_dict['Age (in yr)'] = generate_random_age(int(original_dict['Age (in yr)']))
        modified_dict['Weight (in kg)'] = generate_weight()
        modified_dict['Antibiotic in Past'] =  generate_antibiotics_in_past()
        modified_dict['No. of Days Hospitalized'] =  generate_num_day_hospitalized(2, 10)
        modified_dict['Culture SensitIVity Test'] = generate_culture_sensitivity_test()
        modified_dict = generate_antibiotic_groups(modified_dict)
        
        ################################################
        print(modified_dict)
        
        patient_data = patient_data.append(modified_dict, ignore_index=True)
        break
    break
        

KeyError: '(Antibiotic 1) Generic Name'